# Model V Galaxy Classifier Improved (Spiral, Elliptical, Odd objects)

**Most of this code is made by the authors of the paper of Ghaderi et al. (2025) (https://iopscience.iop.org/article/10.3847/1538-4365/ada8ab) and taken from the GitHub Repository: https://github.com/hmddev1/machine_learning_for_morphological_galaxy_classification**

**This notebook classifies the images of galaxies to spiral and elliptical galaxies and odd objects using the Vision Transformer and VGG16 (Model V).**

**Purpose of this notebook: Improve Model V using seeding and cross-validation**

In [ ]:
import os
path = '/content/drive/Shared drives/DLP Project/Project/Models/Galaxy Models/Looping each Model'
os.chdir(path)
%run imports.py
%matplotlib inline
import plotting
roc_curves = {}

# Paths to images
spath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_spiral'
epath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_elliptical'
opath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_odd'

# Default image size and zernike order.
image_size = 200
zernike_order = 45

# Loading the ZMs and concatenating to a consolidated dataset
spiral_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/spiral_zms.csv')
elliptical_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/elliptical_zms.csv')
odd_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/odd_zms.csv')

spiral_data.drop("Unnamed: 0", axis = 1, inplace = True)
elliptical_data.drop("Unnamed: 0", axis = 1, inplace = True)
odd_data.drop("Unnamed: 0", axis = 1, inplace = True)

all_zm_data = np.concatenate([spiral_data, elliptical_data, odd_data])
np.shape(all_zm_data)

spiral_label = [0] * len(spiral_data)
elliptical_label = [1] * len(elliptical_data)
odd_label = [2] * len(odd_data)

all_labels = spiral_label + elliptical_label + odd_label
len(all_labels)

In [ ]:
def load_galaxy_images(data_dir, target_size):
        """
        Loads, resizes, and processes all JPG images from the specified directory.

        Parameters:
        data_dir (str): The directory containing the JPG images to be processed.
        target_size (tuple): The target size for resizing the images, specified as (width, height).

        Returns:
        list: A list of PIL Image objects, each representing a resized and processed image.

        The function performs the following steps:
        1. Lists all JPG image files in the specified directory.
        2. Reads each image using OpenCV.
        3. Resizes each image to the specified target size.
        4. Scales the pixel values and converts the image to a format compatible with PIL.
        5. Converts each resized image to a PIL Image object.
        6. Appends each PIL Image object to a list.
        7. Returns the list of PIL Image objects.
        """

        all_images = []

        file_path = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]

        for img in file_path:
            image = cv2.imread(img)
            resized_images=cv2.resize(image, target_size)
            resized_images = (resized_images * 255).astype(np.uint8)
            pil_images = Image.fromarray(resized_images)
            all_images.append(pil_images)

        return all_images

sp_dir = '/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_spiral'
el_dir = '/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_elliptical'
odd_dir = '/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_odd'

image_size = 200

sp_img = load_galaxy_images(sp_dir, target_size=(image_size,image_size))
el_img = load_galaxy_images(el_dir, target_size=(image_size,image_size))
odd_img = load_galaxy_images(odd_dir, target_size=(image_size,image_size))

# Due to lack of computational power, we will use a subset of the data (6000 galaxies) while keeping the same ratios between categories.
sp_img = sp_img[:3136]
el_img = el_img[:2082]
odd_img = odd_img[:782]

all_data = sp_img + el_img + odd_img

# Labels
label_s = [0] * len(sp_img)
label_e = [1] * len(el_img)
label_o = [2] * len(odd_img)

all_labels = label_s + label_e + label_o
len(all_labels)


# transforms for training data
train_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.RandomRotation(90),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0), ratio=(0.99, 1.01)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])


# transforms for test data
test_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])



In [ ]:
# Set a fixed seed for reproducibility
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

# Cross Validation Setup using StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed_value)

metrics_list = []
fold = 1

for train_index, test_index in skf.split(np.zeros(len(all_labels)), all_labels):
    print(f"Fold {fold}...")

    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), test_size=0.25, shuffle=True, random_state=None)
     y_train_encoded = to_categorical(y_train, num_classes=3)

    # Transformer for training data

    transformed_X_train = []
    for img in X_train:
        transformed_img = train_transform(img)
        new_img = np.transpose(transformed_img.numpy(), (1, 2, 0))
        transformed_X_train.append(new_img)

    # Transformer for testing data
    transformed_X_test = []
    for img in X_test:
        transformed_img = test_transform(img)
        new_img = np.transpose(transformed_img.numpy(), (1, 2, 0))
        transformed_X_test.append(new_img)

    class_weights = {
        0: len(all_data) / (3 * len(spiral_data)),
        1: len(all_data) / (3 * len(elliptical_data)),
        2: len(all_data) / (3 * len(odd_data))}

    batch_size = 64
    epochs_num = 30

    # Define the pretrained VGG16
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
    x = Flatten()(base_model.output)
    x = Dense(64, activation='relu')(x)
    output = Dense(3, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    history = model.fit(
        np.array(transformed_X_train), y_train_encoded,
        batch_size=batch_size,
        epochs=epochs_num,
        verbose=1,
        callbacks=[es],
        class_weight=class_weights,
        validation_split=0.1)

    y_pred = model.predict(np.array(transformed_X_test))
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred_labels)

    # Performance metrics (per-class)
    recall_per_class = recall_score(y_test, y_pred_labels, average=None)
    precision_per_class = precision_score(y_test, y_pred_labels, average=None)
    f1_per_class = f1_score(y_test, y_pred_labels, average=None)
    accuracy = accuracy_score(y_test, y_pred_labels) # overall

    tss_per_class = {}
    for i, class_name in enumerate(['Spiral', 'Elliptical', 'Odd']):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = np.sum(cm) - (tp + fn + fp)
        tss_per_class[class_name] = (tp / (tp + fn + 1e-6)) - (fp / (fp + tn + 1e-6))

    metrics_list.append({
        'Fold': fold,
        'Recall per Class': recall_per_class,
        'Precision per Class': precision_per_class,
        'F1 per Class': f1_per_class,
        'Accuracy': accuracy,
        'TSS per Class': tss_per_class
    })

    fold += 1

for result in metrics_list:
    print(result)

Fold 1...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 77s 738ms/step - accuracy: 0.3481 - loss: 1.5175 - val_accuracy: 0.1600 - val_loss: 1.1414
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.4269 - loss: 0.5233 - val_accuracy: 0.6022 - val_loss: 0.8059
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.6576 - loss: 0.4073 - val_accuracy: 0.7489 - val_loss: 0.6681
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 384ms/step - accuracy: 0.6757 - loss: 0.4273 - val_accuracy: 0.6978 - val_loss: 0.7925
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.7195 - loss: 0.4000 - val_accuracy: 0.7244 - val_loss: 0.7178
Epoch 6/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 384ms/step - accuracy: 0.7580 - loss: 0.3340 - val_accuracy: 0.7533 - val_loss: 0.6939
Epoch 7/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.8040 - loss: 0.3044 - val_accuracy: 0.7511 - val_loss: 1.2803
Epoch 8/30
64/64 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Running the same code independently, due to limited computational resources.

# Set a fixed seed for reproducibility
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

# Cross Validation Setup using StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed_value)

metrics_list = []
fold = 1

for train_index, test_index in skf.split(np.zeros(len(all_labels)), all_labels):
    print(f"Fold {fold}...")

    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), test_size=0.25, shuffle=True, random_state=None)
     y_train_encoded = to_categorical(y_train, num_classes=3)

    # Transformer for training data

    transformed_X_train = []
    for img in X_train:
        transformed_img = train_transform(img)
        new_img = np.transpose(transformed_img.numpy(), (1, 2, 0))
        transformed_X_train.append(new_img)

    # Transformer for testing data
    transformed_X_test = []
    for img in X_test:
        transformed_img = test_transform(img)
        new_img = np.transpose(transformed_img.numpy(), (1, 2, 0))
        transformed_X_test.append(new_img)

    class_weights = {
        0: len(all_data) / (3 * len(spiral_data)),
        1: len(all_data) / (3 * len(elliptical_data)),
        2: len(all_data) / (3 * len(odd_data))}

    batch_size = 64
    epochs_num = 30

    # Define the pretrained VGG16
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
    x = Flatten()(base_model.output)
    x = Dense(64, activation='relu')(x)
    output = Dense(3, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    history = model.fit(
        np.array(transformed_X_train), y_train_encoded,
        batch_size=batch_size,
        epochs=epochs_num,
        verbose=1,
        callbacks=[es],
        class_weight=class_weights,
        validation_split=0.1)

    y_pred = model.predict(np.array(transformed_X_test))
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred_labels)

    # Performance metrics (per-class)
    recall_per_class = recall_score(y_test, y_pred_labels, average=None)
    precision_per_class = precision_score(y_test, y_pred_labels, average=None)
    f1_per_class = f1_score(y_test, y_pred_labels, average=None)
    accuracy = accuracy_score(y_test, y_pred_labels) # overall

    tss_per_class = {}
    for i, class_name in enumerate(['Spiral', 'Elliptical', 'Odd']):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = np.sum(cm) - (tp + fn + fp)
        tss_per_class[class_name] = (tp / (tp + fn + 1e-6)) - (fp / (fp + tn + 1e-6))

    metrics_list.append({
        'Fold': fold,
        'Recall per Class': recall_per_class,
        'Precision per Class': precision_per_class,
        'F1 per Class': f1_per_class,
        'Accuracy': accuracy,
        'TSS per Class': tss_per_class
    })

    fold += 1

for result in metrics_list:
    print(result)

Fold 1...
Epoch 1/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 32s 429ms/step - accuracy: 0.3576 - loss: 2.2756 - val_accuracy: 0.4644 - val_loss: 1.0817
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.5183 - loss: 0.5519 - val_accuracy: 0.3578 - val_loss: 1.0463
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.4408 - loss: 0.4870 - val_accuracy: 0.6400 - val_loss: 0.7806
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.6338 - loss: 0.4106 - val_accuracy: 0.7533 - val_loss: 0.6822
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 386ms/step - accuracy: 0.6963 - loss: 0.3853 - val_accuracy: 0.7644 - val_loss: 0.6680
Epoch 6/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.7278 - loss: 0.3489 - val_accuracy: 0.7844 - val_loss: 0.6013
Epoch 7/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 386ms/step - accuracy: 0.7680 - loss: 0.3240 - val_accuracy: 0.8000 - val_loss: 0.5941
Epoch 8/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.7786 - loss: 0.3074 